In [1]:
import numpy as np
import copy
from shapely.geometry.polygon import LinearRing, Polygon
from shapely.geometry import Point
import random
from shapely import affinity
import matplotlib.pyplot as plt
%matplotlib notebook
import time

In [2]:
class Room():        
    def __init__(self, shape):
        self.shape = shape
        self.polygon = Polygon(np.array(shape, dtype=np.float64))

In [3]:
class FurnitureItem(): 
    def __init__(self, unitcost, shape):
        self.shape = shape
        self.unit_cost = unitcost
        self.polygon = Polygon(np.array(shape, dtype=np.float64))
        self.total_cost = np.float64((np.float64(self.unit_cost) * self.polygon.area))
        self.sorting = (self.polygon, self.polygon.area)

In [4]:
class Problem():
    def __init__(self, room, furniture):
        self.room = room
        self.furniture = furniture

In [5]:
problems = []
with open('problems.rfp') as f:
    for line in f:
        furnitureList = []
        problem = line.replace(" ","").split("#")

        room_shape = problem[0].split(":")[1]
        furniture = problem[1].split(";")

        room = Room([tuple(np.float64(i) for i in el.strip('()').split(',')) for el in room_shape.split('),(')])

        for furnitureItem in furniture:
            parse_furniture = furnitureItem.strip().split(":")
            unit_cost = parse_furniture[0]
            shape = [tuple(np.float64(i) for i in el.strip('()').split(',')) for el in parse_furniture[1].split('),(')]
            furnitureItem = FurnitureItem(unit_cost, shape)
            furnitureList.append(furnitureItem)
        problems.append(Problem(room,furnitureList))

In [6]:
def plot(poly):
    x,y = poly.exterior.xy
    plt.plot(x, y, color='#6699cc', alpha=0.7, linewidth=3, solid_capstyle='round', zorder=2)
    plt.show()

In [7]:
def random_points_within(poly, num_points):
    min_x, min_y, max_x, max_y = poly.bounds
    points = []
    while len(points) < num_points:
        random_point = Point([random.uniform(min_x, max_x), random.uniform(min_y, max_y)])
        if (random_point.within(poly)):
            points.append(random_point)
    return points

In [ ]:
solution = []
room_2 = problems[10].room.polygon
shapes = copy.copy(problems[10].furniture)
initial_area = room_2.area
while (room_2.area) > initial_area*0.7:
    for j in shapes:
        x,y = j.polygon.exterior.xy
        if len(x) >= 5:
            points = random_points_within(room_2,1)[0]
            polygon = affinity.translate(j.polygon, points.x, points.y)
            polygon = affinity.rotate(polygon, random.uniform(0,180),origin="centroid")
            if room_2.contains(polygon):
                room_2 = room_2.difference(polygon)
                x,y = polygon.exterior.xy
                solution.append(list(zip(*(x,y))))
                shapes.remove(j)
                print(len(shapes), (room_2.area))
        else:
            shapes.remove(j)

room_2.area

493 91.24999999999991
482 90.24999999999991
481 89.24999999999991
480 88.24999999999991
479 87.24999999999991
478 86.24999999999991
477 85.24999999999991
476 84.24999999999991
475 83.24999999999991
474 81.99999999999991
473 80.99999999999991
472 79.99999999999991
471 78.99999999999991
470 77.99999999999991
469 76.99999999999991
468 75.99999999999991
467 74.99999999999991


In [ ]:
output = ""
for x in solution:
    output = output + str(x[:-1]).replace('[','').replace(']','') + "; "
print(output)

In [17]:
x,y = problems[0].furniture[0].polygon.exterior.xy

In [18]:
len(x)

5